# **Dia 4**

Uma das aplicações que mais interessam empresas de todos os setores são os sistemas de recomendação: aplicações que aprendem, a partir de dados históricos, como sugerir os melhores itens para clientes a fim de maximizar alguma métrica de negócio, como faturamento, taxa de conversão, dentre outras.

E é exatamente sobre isso que eu vou falar no desafio de hoje!

O ***MovieLens*** é um dataset clássico usado em problemas de sistema de recomendação. Ele é usado até em artigos científicos para validar novos tipos de algoritmos de recomendação. A partir dele criaremos um sistema de recomendação que dado um filme, recomendaremos 5 filmes que mais se assemelham a ele.

Para este dia utilizaremos as seguintes bibliotecas:

- Pandas
- Numpy
- Scikit-Learn
- Joblib

Utlizaremos cada uma para a leitura e tratamento dos dados, outra para operações matemáticas, depois para pré-processamento dos dados e constução do nosso modelo, e o último para podermos salvar o modelo que criarmos.

In [1]:
import pandas as pd
import numpy as np
import joblib

## **Tratamento dos Dados**

In [2]:
df_filmes = pd.read_csv('../DS_Dia_4/u.item', sep='|', names=['movie id', 'movie title', 'release date', 'video release date',
                 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 
                 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
                 encoding='UTF-8')

df_filmes = df_filmes.set_index(keys='movie id')
df_filmes.head()

,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Como de início, sempre devemos verificar e se necessário tratar nossos dados, porque muitas vezes podem vir sujos e com falta de informação, afetando então nosso modelo. Note que estamos deixando o `movie id` como sendo nosso index por enquanto, vamos apenas verificar a existencia de filmes duplicados após isso resetaremos o index.

De início, queremos avaliar um filme com base em seu nome e suas características, iremos retirar então as colunas `release date`, `video release date` e `IMDb URL` de noso DataFrame. Após isso utilizaremos a função `info()` para ver se temos dados nulos e se o tipo de cada dado está correto, em seguida utilizaremos as funções `duplicated()` e `sum()` para verificar se temos filmes duplicados.

In [3]:
df_filmes = df_filmes.drop(columns=['release date', 'video release date', 'IMDb URL'])
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1682 entries, 1 to 1682
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie title  1682 non-null   object
 1   unknown      1682 non-null   int64 
 2   Action       1682 non-null   int64 
 3   Adventure    1682 non-null   int64 
 4   Animation    1682 non-null   int64 
 5   Childrens    1682 non-null   int64 
 6   Comedy       1682 non-null   int64 
 7   Crime        1682 non-null   int64 
 8   Documentary  1682 non-null   int64 
 9   Drama        1682 non-null   int64 
 10  Fantasy      1682 non-null   int64 
 11  Film-Noir    1682 non-null   int64 
 12  Horror       1682 non-null   int64 
 13  Musical      1682 non-null   int64 
 14  Mystery      1682 non-null   int64 
 15  Romance      1682 non-null   int64 
 16  Sci-Fi       1682 non-null   int64 
 17  Thriller     1682 non-null   int64 
 18  War          1682 non-null   int64 
 19  Western      1682 non-null   int

In [4]:
df_filmes.duplicated().sum()

np.int64(18)

Como uma breve visualização, vemos que não temos dados nulos ou problemas com tipos, porém vemos que temos 18 filmes duplicados. Manteremos por enquanto porque estaremos realizando uma verificação com as notas de usuários, dessa forma veremos como cada duplicata tem em quantidade de avaliação de usuário.

Faremos mais importações de mais datasets, um com os usuários e outro com suas avaliações em cada filme. Faremos mais uma vez o procedimento de tratar os dados.

In [5]:
df_usuarios = pd.read_csv('../DS_Dia_4/u.user', sep='|', names=['user id', 'age', 'gender', 'occupation', 'zip code'], encoding='ISO-8859-1')
df_usuarios.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
df_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user id     943 non-null    int64 
 1   age         943 non-null    int64 
 2   gender      943 non-null    object
 3   occupation  943 non-null    object
 4   zip code    943 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB


In [7]:
df_usuarios.duplicated().sum()

np.int64(0)

Apenas verificaremos as colunas gênero e ocupação para descobrir quais valores categóricos temos.

In [8]:
df_usuarios['gender'].unique()

array(['M', 'F'], dtype=object)

In [9]:
df_usuarios['occupation'].unique()

array(['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor'],
      dtype=object)

Para o dataset dos usuários não foi encontrado nenhum problema. Apenas retiraremos a coluna `zip code` que informa a localização geográfica da pessoa.

In [10]:
df_usuarios = df_usuarios.drop('zip code', axis=1)
df_usuarios.head()

,user id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


Seguiremos para o dataset das avaliações.

In [11]:
df_avaliacoes = pd.read_csv('../DS_Dia_4/u.data', sep='\t', names=['user id', 'item id', 'rating', 'timestamp'], encoding='ISO-8859-1')
df_avaliacoes

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [12]:
df_avaliacoes = df_avaliacoes.drop('timestamp', axis=1)
df_avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user id  100000 non-null  int64
 1   item id  100000 non-null  int64
 2   rating   100000 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [13]:
df_avaliacoes.duplicated().sum()

np.int64(0)

Novamente nenhum problema para este dataset.

Apenas verificaremos como estão as avalições para filmes duplicados.

In [14]:
df_filmes[df_filmes.duplicated() == True].head()

,movie title,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,,
268,Chasing Amy (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
303,Ulee's Gold (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
348,Desperate Measures (1998),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
500,Fly Away Home (1996),0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
670,Body Snatchers (1993),0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0


In [15]:
df_filmes[df_filmes['movie title'] == "Chasing Amy (1997)"]

,movie title,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,,
246,Chasing Amy (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
268,Chasing Amy (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [16]:
df_avaliacoes[df_avaliacoes['item id'] == 246].shape

(124, 3)

In [17]:
df_avaliacoes[df_avaliacoes['item id'] == 268].shape

(255, 3)

Embora existam 18 filmes com títulos semelhantes e IDs distintos no ***MovieLens***, cada um possui um volume significativo de avaliações (>100). Para evitar fusões incorretas e perda de informação, os filmes serão mantidos como itens separados.

Faremos agora o reset do index do DataFrame *df_filmes*, copiaremos para um DataFrame novo para preservar algumas caracteríticas e retiraremos a coluna do título de cada filme que não será necessária no momento.

In [18]:
df_filmes = df_filmes.reset_index()
df_filmes_final = df_filmes.drop('movie title', axis=1).copy()
df_filmes_final.head()

,movie id,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


## **Pré-Processamento**

A partir de agora estaremos preparando os nossos dados para aplicar um modelo de machine learning, com isso estaremos separando colunas categóricas e juntando todos os DataFrames.

In [19]:
df_user_aval = pd.merge(df_avaliacoes, df_usuarios, on='user id')
df_user_aval.head()

,user id,item id,rating,age,gender,occupation
0,196,242,3,49,M,writer
1,186,302,3,39,F,executive
2,22,377,1,25,M,writer
3,244,51,2,28,M,technician
4,166,346,1,47,M,educator


Consideraremos na coluna gênero como sendo 1 para gênero masculino e 0 para feminino.

In [20]:
df_user_aval['gender'] = df_user_aval['gender'].replace({'M': 1, 'F': 0})
df_user_aval.head()

/tmp/ipykernel_88691/1438402274.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_user_aval['gender'] = df_user_aval['gender'].replace({'M': 1, 'F': 0})


,user id,item id,rating,age,gender,occupation
0,196,242,3,49,1,writer
1,186,302,3,39,0,executive
2,22,377,1,25,1,writer
3,244,51,2,28,1,technician
4,166,346,1,47,1,educator


Estaremos novamente juntando mais colunas, só que agora do DataFrame dos filmes.

In [21]:
df_filmes_final = df_filmes_final.rename(columns={'movie id': 'item id'})

df_treino = df_user_aval.copy()
df_treino = pd.merge(df_treino, df_filmes_final, on='item id')
df_treino = df_treino.drop(['user id', 'occupation',], axis=1)

df_treino = df_treino.groupby('item id').mean()

df_treino

,rating,age,gender,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item id,,,,,,,,,,,,,,,,,,,,,
1,3.878319,31.955752,0.736726,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.206107,29.312977,0.854962,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3.033333,27.011111,0.822222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,3.550239,32.593301,0.784689,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3.302326,29.930233,0.744186,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,1.000000,17.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,3.000000,17.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1680,2.000000,17.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Retiramos a coluna ocupaçao para não sobrecarregar o treinamento, alías, são mais de 20 tipos de ocuapação que no final não seriam tão importantes para o treinamento do modelo.
Vemos também que ao todo temos 22 colunas, isso não é bom para nosso modelo uma vez que são 22 dimensões para seu treinamento o que aumenta o tempo de treinamento. Para isso aplicaremos técnicas de pré-processamento para reduzir a dimensionalidade em uma quantidade que já explique bem os dados que temos.

Vamos aplicar o **PCA** para reduzir a dimensão, mas antes disso, vamos aplicar o **StandardScaler** para escalar os dados e normalizar eles, o **PCA** é sensível a dados não escalados.

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [23]:
np.random.seed(3165)

scaler = StandardScaler()
pca = PCA(n_components=22, random_state=3165)

treino_scaler = scaler.fit_transform(df_treino)

treino_pca = pca.fit(treino_scaler)

In [24]:
validacao = 0

for i, valor in enumerate(treino_pca.explained_variance_ratio_):
    validacao += valor
    if(validacao >= 0.70):
        print(f'{i+1} elementos explicam {validacao*100:.2f}% dos dados')
        break


12 elementos explicam 71.09% dos dados


Vemos que conseguimos reduzir a dimensão de nossos dados para 12, com um explicação de mais de 70% dos dados.

Vamos utilizar essa quantidade para treinar nosso modelo, utilizaremos o **KMeans** como modelo de recomendação principal.

## **Modelo de Recomendação**

In [25]:
from sklearn.cluster import KMeans

In [26]:
pca_final = PCA(n_components=12, random_state=3165)

treino_final = pca_final.fit_transform(treino_scaler)
df_treino_final = pd.DataFrame(data=treino_final, columns=pca_final.get_feature_names_out())

Para verificar quantos clusteres devemos usar, utilizaremos a silhueta para medir quão bem cada ponto se encaixa no seu cluster em comparação aos outros clusters e a inércia que é a soma das distâncias quadráticas entre cada ponto e o centroide do seu cluster. Dessa forma teremos uma melhor indicação de em qual direção seguir.

Segue a função abaixo para nossa análise.

In [27]:
from sklearn.metrics import silhouette_score

def avaliacao(dados):
  inercia = []
  silhueta = []

  for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, random_state=3165)
    kmeans.fit(dados)
    inercia.append(f'k={k} - ' + str(kmeans.inertia_))
    silhueta.append(f'k={k} - '+ str(silhouette_score(dados, kmeans.predict(dados))))
  
  return silhueta, inercia

Vamos fazer as medições para ver o quão bem está a previsão.

In [28]:
silhueta, inercia = avaliacao(df_treino_final)
silhueta

['k=2 - 0.13564241818561318',
 'k=3 - 0.1551789175585974',
 'k=4 - 0.18219506699720767',
 'k=5 - 0.2100851028666265',
 'k=6 - 0.22634723393750011',
 'k=7 - 0.24488520847812598',
 'k=8 - 0.27563752721667323',
 'k=9 - 0.28985892086579373',
 'k=10 - 0.2927728474368075',
 'k=11 - 0.2976386675028365',
 'k=12 - 0.31841243322357343',
 'k=13 - 0.34386050753729525',
 'k=14 - 0.35194721541160046',
 'k=15 - 0.35851720658863084',
 'k=16 - 0.351110053649327',
 'k=17 - 0.36808514206741577',
 'k=18 - 0.35871786842356923',
 'k=19 - 0.3626794358564933']

In [29]:
inercia

['k=2 - 24120.804552726153',
 'k=3 - 22014.238752990852',
 'k=4 - 19972.54202846166',
 'k=5 - 18264.26910280713',
 'k=6 - 16602.519759511626',
 'k=7 - 15596.161146176815',
 'k=8 - 13711.94436301709',
 'k=9 - 12438.520615765505',
 'k=10 - 10761.55597221018',
 'k=11 - 10040.133661301581',
 'k=12 - 8697.167938770795',
 'k=13 - 8129.033442522223',
 'k=14 - 7637.12617265159',
 'k=15 - 6917.020138125079',
 'k=16 - 6550.618119624221',
 'k=17 - 6066.132559688347',
 'k=18 - 5874.98848272381',
 'k=19 - 5625.256895287082']

Vemos em geral a silhueta que deveria está próxima de 1 para indicar que os pontos centrais estão bem alocados, não deram um resultado satisfatório, sendo necessário uma maior quantidade talvez, mas devido a termos 1600 filmes aproximadamente, aumentar a quantidade de clusteres dividiria muito e não serveria realizar uma classificação e recomendação adequadas.

Utilizaremos 9 clusteres, pois se usassemos mais, cairíamos no caso de muitos filmes dividos entre clusteres e como queremos recomendar 5 filmes, um cluster com menos de 10 filmes já seria muito pouco. 9 é um número bem razoável para nosso modelo.

In [30]:
modelo_kmeans = KMeans(n_clusters=9, random_state=3165)

modelo_kmeans.fit(df_treino_final)

df_treino_final['cluster'] = modelo_kmeans.labels_

clusteres = df_treino_final.groupby('cluster').mean()

clusteres.T

cluster,0,1,2,3,4,5,6,7,8
pca0,-1.065590,-0.193085,2.892808,1.804306,0.150614,0.039172,0.800081,-0.819445,3.975722
pca1,-0.081738,0.821677,4.304198,-1.167166,-0.183138,-2.155345,-0.673894,0.195555,2.499535
pca2,0.410599,-0.693677,2.543104,-0.017867,0.117120,0.896387,-1.662460,0.113431,0.996631
pca3,-0.305860,0.067234,2.150415,-0.533738,-1.281031,3.320927,0.303868,-0.126580,-0.691314
pca4,-0.746111,1.120569,-0.287890,-0.224764,1.448370,0.216018,0.481063,0.080391,-2.718744
pca5,-0.018224,0.526779,-1.309333,0.197476,-1.132834,1.126902,-2.145314,-3.002942,2.552384
pca6,-0.271872,0.145784,-2.191127,-0.276555,1.501835,0.610241,-0.553716,2.851415,4.416778
pca7,0.145507,0.008942,0.116899,-0.020450,5.009162,-0.546589,-0.509271,-2.763896,-0.233110
pca8,0.174802,0.038971,-0.678773,-0.482680,-2.726572,-0.295698,2.329001,-2.676467,2.786959
pca9,0.015183,0.106571,-0.132601,-0.028293,-2.369547,0.376423,-0.337285,0.164436,0.570976


In [31]:
df_treino_final.groupby('cluster')[['cluster']].value_counts()

cluster
0    635
1    453
2     42
3    287
4     27
5     77
6     89
7     50
8     22
Name: count, dtype: int64

Acima vemos a tabela das médias dos valores entre cada cluster, mais para frente tornaremos esses números positivos para crirar nossa função recomendadora.

Agora a última parte mostra quantos filmes está em cada cluster. Vemos que conseguimos no mínimo 22 filmes em um cluster, o que é um resultado satisfatório.

Criaremos um pipeline do nosso modelo e salvaremos com a biblioteca **joblib** para a reutilização do modelo.

In [32]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=12, random_state=3165)),
            ('kmeans', KMeans(n_clusters=9, random_state=3165))
            ])

modelo_recomendacao = pipeline.fit(df_treino)

In [33]:
joblib.dump(modelo_recomendacao, 'modelo_recomendacao.pkl')

['modelo_recomendacao.pkl']

In [34]:
df_treino_final

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,cluster
0,2.573730,4.386668,2.148197,2.072672,0.344838,-0.788673,-0.842704,0.411267,-0.010954,-0.077238,0.167871,0.922903,2
1,3.092375,-2.184329,0.288493,-0.813999,-0.287884,0.409395,-0.269105,0.038338,-1.017974,-0.164892,0.229307,-1.337936,3
2,0.584768,-1.120218,-0.563476,0.895046,-0.047116,-0.315045,-0.516393,-0.165277,-0.265185,-0.022117,-0.235280,-0.300827,3
3,-0.013080,-0.388273,-0.021295,-0.965972,0.577530,0.344553,-0.197290,0.782066,-0.009316,-0.113563,0.670912,-0.448841,1
4,-0.494945,-2.070419,0.317796,1.799348,-1.528463,0.361368,-0.514645,1.203933,-0.016654,-0.011854,1.980529,-0.460318,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.124270,0.105917,-2.320087,-0.090317,-1.294155,-0.342341,-0.773747,0.883985,-0.058449,0.186423,0.466276,0.278238,0
1678,0.738627,-1.015455,-1.124570,0.672272,0.803219,1.011297,-1.430533,-0.693258,-0.577434,0.231304,-0.307155,-0.844967,3
1679,-0.402815,0.185493,-1.545175,-0.448815,-0.275264,1.001687,-1.320154,0.021204,-0.185775,0.327159,0.094984,-0.499609,1
1680,0.172037,0.409698,-0.821754,0.138005,1.671574,-0.053612,0.422553,0.586377,0.420974,0.195655,0.680108,0.157594,1


In [ ]:
teste = modelo_recomendacao[:-1].transform(df_treino)
teste = pd.DataFrame(data=teste, columns=pca_final.get_feature_names_out())

teste['cluster'] = modelo_kmeans.labels_

modelo_kmeans.feature_names_in_['pca']
teste

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,cluster
0,2.573730,4.386668,2.148197,2.072672,0.344838,-0.788673,-0.842704,0.411267,-0.010954,-0.077238,0.167871,0.922903,2
1,3.092375,-2.184329,0.288493,-0.813999,-0.287884,0.409395,-0.269105,0.038338,-1.017974,-0.164892,0.229307,-1.337936,3
2,0.584768,-1.120218,-0.563476,0.895046,-0.047116,-0.315045,-0.516393,-0.165277,-0.265185,-0.022117,-0.235280,-0.300827,3
3,-0.013080,-0.388273,-0.021295,-0.965972,0.577530,0.344553,-0.197290,0.782066,-0.009316,-0.113563,0.670912,-0.448841,1
4,-0.494945,-2.070419,0.317796,1.799348,-1.528463,0.361368,-0.514645,1.203933,-0.016654,-0.011854,1.980529,-0.460318,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.124270,0.105917,-2.320087,-0.090317,-1.294155,-0.342341,-0.773747,0.883985,-0.058449,0.186423,0.466276,0.278238,0
1678,0.738627,-1.015455,-1.124570,0.672272,0.803219,1.011297,-1.430533,-0.693258,-0.577434,0.231304,-0.307155,-0.844967,3
1679,-0.402815,0.185493,-1.545175,-0.448815,-0.275264,1.001687,-1.320154,0.021204,-0.185775,0.327159,0.094984,-0.499609,1
1680,0.172037,0.409698,-0.821754,0.138005,1.671574,-0.053612,0.422553,0.586377,0.420974,0.195655,0.680108,0.157594,1


## **Função Recomendadora**

A partir de agora construiremos nossa função recomendadora, separaremos em algumas partes para isolar os processos.

De começo vamos criar uma função capaz de calcular a distancia euclidiana entre um filme escolhido e outros filmes do mesmo cluster do escolhido e ver quais são os mais próximos.

In [36]:
def mais_proximos(df_avaliacao, cluster_index, id_filme):
    # Pega os filmes do cluster especifico retirando o filme que faremos a medicao e adiciona a coluna distancia
    filmes_cluster = df_avaliacao[df_avaliacao['cluster'] == cluster_index].copy()
    filmes_cluster = filmes_cluster[filmes_cluster["item id"] != id_filme]
    filmes_cluster = filmes_cluster.reset_index(drop=True)
    filmes_cluster['distancia'] = 0

    for linha in range(len(filmes_cluster['item id'])):
        # Pega o filme que faremos a medicao
        filme = df_avaliacao[df_avaliacao['item id'] == id_filme].copy()
        filme = filme.reset_index(drop=True)

        # Calcula a distancia euclidiana do nosso filme com o restante
        valor = []
        for coluna in filme.columns:
            if coluna not in ['item id', 'cluster', 'distancia']:
                valor.append(pow(filme.loc[0, coluna] - filmes_cluster.loc[linha, coluna], 2))

        filmes_cluster.loc[linha, 'distancia'] = np.sqrt(np.sum(valor))
    
    # Devolve os filmes em ordem crescente do mais proximos em ids
    filmes_cluster = filmes_cluster.sort_values('distancia').reset_index(drop=True)
    top_filmes = []
    for linha in range(len(filmes_cluster['item id'])):
        top_filmes.append(filmes_cluster.loc[linha, 'item id'])

    return filmes_cluster, top_filmes

Logo após criaremos uma outra função capaz de escolher o "melhor" filme de um usuário, isto é, o filme que o úsuario pode ter mais se familizarizado e retornaremos o *id* do filme.

In [37]:
def melhor_filme_cluster(cluster_index, usuario, df_filmes):
    # Separa o usuário com base no cluster escolhido
    usuario_modificado = usuario[usuario['cluster'] == cluster_index].copy()

    # Preparação para escolher o melhor filme
    somatorio = usuario_modificado.drop(['age','item id', 'gender'], axis=1).copy()
    somatorio['resultado'] = 0

    for linha in range(len(somatorio['rating'])):
        # Pega o id de cada filme e multiplica a coluna das avaliações
        # com base na média geral do filme
        id_filme = usuario_modificado.iloc[linha]['item id']
        somatorio.iloc[linha]['rating'] *= df_filmes.loc[id_filme, 'rating']

        # Soma todas as colunas de devolve o resultado
        somatorio.iloc[linha]['resultado'] = np.sum(somatorio.iloc[linha])

    # Implementa o resultado no DataFrame do usuário e escolhe o id do melhor filme
    # com base no maior resultado
    usuario_modificado['resultado'] = somatorio['resultado']
    id_melhor_filme = usuario_modificado.sort_values('resultado', ascending=False).iloc[0]['item id']

    return id_melhor_filme
        

Por último faremos a lógica da preparação do usuário e dos filmes que ele assistiu, então pegaremos o cluster mais "bem avaliado" e tomaremos uma ordem do melhor para o pior, com isso tentaremos pegar 5 filmes que o usuario não assitiu e classificados como possíveis melhores recomendações para o usuário.

In [38]:
def recomendacao(df_usuario, df_filmes, df_avaliacao, id_usuario):
    # Prepara os filmes para a classificação com relação ao usuário
    filmes = df_filmes.reset_index().copy()
    filmes = filmes.drop(['rating', 'age', 'gender'], axis=1)

    # Faz a predicao dos clusteres de cada filme avaliado e agrupa no DataFrame do usuário
    usuario = df_usuario[df_usuario['user id'] == id_usuario].copy()
    usuario = pd.merge(usuario, filmes, on='item id')
    usuario = usuario.drop(['user id', 'occupation'], axis=1)
    
    clusteres = modelo_recomendacao.predict(usuario.drop(['item id'], axis=1))
    usuario['cluster'] = clusteres

    # Pega os melhores clusteres mais bem avaliados com base na quantidade e soma dos mais
    # bem avaliados filmes feitos pelo usuário com base na multiplicação deles
    melhor_cluster = usuario.drop(['age','gender', 'item id'], axis=1).copy()
    mais_assistidos = melhor_cluster[['cluster']].groupby('cluster').value_counts()
    melhor_cluster = melhor_cluster.groupby('cluster').sum()
    melhor_cluster['rating'] = melhor_cluster['rating'] * mais_assistidos

    # Preparação do próximo passo
    df_avaliacao['item id'] = df_filmes.index

    # Procura o cluster com pelo menos 5 filmes potenciais para recomendar ao usuário
    for incremento in melhor_cluster.sort_values('rating', ascending=False).index:
        # Pega o index dos melhores clusteres na ordem decrescente, pega o melhor filme e
        # por último pega o DataFrame e o índice dos filmes mais próximos
        cluster_index = melhor_cluster.sort_values('rating', ascending=False).index[incremento]

        id_melhor_filme = melhor_filme_cluster(cluster_index, usuario, df_filmes)
        filmes_cluster, top_filmes = mais_proximos(df_avaliacao, cluster_index, id_melhor_filme)

        # Se o usário ja viu o filme retira do DataFrame de filmes e se o DataFrame
        # tem 5 ou mais elementos, retorna os 5 filmes mais próximos
        for x in top_filmes:
            if x in usuario['item id']:
                 filmes_cluster = filmes_cluster[filmes_cluster['item id'] != x]
                 
        if len(filmes_cluster >= 5):
            break

    return filmes_cluster['item id'].head()
    

Segue por fim um demonstração de como ficou o resultado final.

Fizemos algumas alterações nos avisos para uma melhor visulização.

In [39]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

filmes_cluster = recomendacao(df_user_aval, df_treino, df_treino_final, 15)

filmes = df_filmes[df_filmes['movie id'].isin(filmes_cluster.to_list())]['movie title'].values

print('\nRecomendação de filmes:')
print(f'1- {filmes[0]}')
print(f'2- {filmes[1]}')
print(f'3- {filmes[2]}')
print(f'4- {filmes[3]}')
print(f'5- {filmes[4]}')


Recomendação de filmes:
1- Diabolique (1996)
2- Kalifornia (1993)
3- Extreme Measures (1996)
4- City Hall (1996)
5- Juror, The (1996)


## **Conclusão**

Neste dia utilizamos o dataset da ***MovieLens*** que apresentou pouquíssimos problemas com o tratamento dos dados, para construir um modelo de recomendação a partir das avaliações realizadas por usuários.

Depois do tratamento, fizemos um pré-processamento para transformar nosso dados para ser mais palpável para o modelo de machine learning. Utilizamos o **StandardScaler** e o **PCA** para normalizar e diminuir a dimensão de nossos dados. Com isso utilizamos o modelo **KMeans** como sendo nosso modelo separando ao todo em 9 clusteres.

Por último criamos nossa função recomendadora utilizando da distância euclidana entre filmes de mesmo cluster.

Apesar de simples, nosso modelo segue com um resultado satisfatório, sendo pouco tendencioso, mas garantindo uma melhor escolha para nosso usuário. No final, o modelo consegue recomendar 5 filmes, sendo simples e reutilizável futuramente para um bom direcionamento de como modelos mais complexos podemo seguir.